----
CC5201-1 Bases de Datos Multimedia.<br>
Genesis Moraga Azócar <br>           
***

# Tarea 3
***

##Parte I:
Computar una nueva función de distancia que combine los
resultados de los algoritmos y analizar los resultados de MAP.


####Carga de datos
**Antes de correr el siguiente código se deben cargar los archivos: algoritmo1.txt, algoritmo2.txt, algoritmo3.txt, query.cla y target.cla.**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Carga de datos algoritmo 1

f1 = open("algoritmo1.txt", "r")
v_alg1=f1.readlines()
f1.close()
alg1=np.zeros((len(v_alg1),96487)) #matriz que guardará los datos del algoritmo

for i in range(len(v_alg1)):
  linea=v_alg1[i]
  y=0
  pos=0
  for j in range(len(linea)):
    if linea[j]==" ":
      alg1[i][y]=linea[pos:j]
      y+=1
      pos=j+1
    if j==len(linea)-1:
      alg1[i][y]=linea[pos:]


#Carga de datos algoritmo 2

f2 = open("algoritmo2.txt", "r")
v_alg2=f2.readlines()
f2.close()
alg2=np.zeros((len(v_alg2),96487)) #matriz que guardará los datos del algoritmo

for i in range(len(v_alg2)):
  linea=v_alg2[i]
  y=0
  pos=0
  for j in range(len(linea)):
    if linea[j]==" ":
      alg2[i][y]=linea[pos:j]
      y+=1
      pos=j+1
    if j==len(linea)-1:
      alg2[i][y]=linea[pos:]


#Carga de datos algoritmo 3

f3 = open("algoritmo3.txt", "r")
v_alg3=f3.readlines()
f3.close()
alg3=np.zeros((len(v_alg3),96487)) #matriz que guardará los datos del algoritmo

for i in range(len(v_alg3)):
  linea=v_alg3[i]
  y=0
  pos=0
  for j in range(len(linea)):
    if linea[j]==" ":
      alg3[i][y]=linea[pos:j]
      y+=1
      pos=j+1
    if j==len(linea)-1:
      alg3[i][y]=linea[pos:]

In [3]:
#Carga de las clases de los objetos query y target
c1 = open("target.cla", "r")
t=c1.readlines()
c1.close()
c2=open("query.cla", "r")
q=c2.readlines()
c2.close()

clase_target= np.zeros(96487)
clase_query= np.zeros(229)

cont=3
objeto=0

for i in range(9):
  line_obj=t[cont]
  if i==8:
    n_obj=int(line_obj[-6:])
  else: 
    n_obj=int(line_obj[-3:])
  for j in range(cont+1,cont+n_obj+1):
    clase_target[objeto]=i
    objeto+=1
  cont+=n_obj+2

cont=3
objeto=0

for i in range(9):
  line_obj=q[cont]
  n_obj=int(line_obj[-3:])
  for j in range(cont+1,cont+n_obj+1):
    clase_query[objeto]=i
    objeto+=1
  cont+=n_obj+2


####Calcular las matrices de distancia combinadas $D_{AB}$, $D_{AC}$ y $D_{BC}$

In [4]:
D_ab=alg1+alg2

D_ac=alg1+alg3

D_bc=alg2+alg3

####Calcular el MAP para cada una de las distancias combinadas.

In [5]:
def rel_numrel(d, query, target):
  ord=np.argsort(d,axis=1)

  relevantes=np.zeros((229,96487))
  for i in range(229):
    for j in range(96487):
      if query[i]==target[ord[i][j]]:
        relevantes[i][j]=1
  
  num_relevantes=np.zeros(229)
  for i in range(229):
    num=0
    for j in range(96487):
      if relevantes[i][j]==1:
        num+=1
    num_relevantes[i]=num

  return relevantes, num_relevantes

In [6]:
relevantes_Dab, num_relevantes_Dab = rel_numrel(D_ab, clase_query, clase_target)
relevantes_Dac, num_relevantes_Dac = rel_numrel(D_ac, clase_query, clase_target)
relevantes_Dbc, num_relevantes_Dbc = rel_numrel(D_bc, clase_query, clase_target)

In [7]:
def recall_precision(relevantes, num_relevantes):
  matriz_precision=np.zeros((229,11))
  matriz_recall=np.zeros((229,11))

  for i in range(229):
    precision = []
    recall = []

    cont = 0
    for j in range(96487):
      if relevantes[i][j]==1:
        cont=cont+1
      precision.append(cont/(j+1))
      recall.append(cont/num_relevantes[i])

      if cont/num_relevantes[i]==1:
        break

    recall = [val*100 for val in recall]
    numBins = 10
    currentRecall = 100 - 100/numBins
    maxPrec = precision[-1]
    precision_recall = np.zeros((numBins+1))
    recall2 = np.zeros((numBins+1))
    pos = numBins
    precision_recall[pos] = maxPrec
    recall2[pos] = 100
    cont = len(precision)-2
    pos = pos - 1

    while cont >=0:
      if recall[cont]>= currentRecall:
        if precision[cont] > maxPrec:
          maxPrec = precision[cont]
        cont = cont - 1
      else:
        precision_recall[pos] = maxPrec
        recall2[pos] = currentRecall
        currentRecall=currentRecall - numBins
        pos = pos - 1

    while pos>=0:
      precision_recall[pos] = maxPrec
      pos = pos-1

    recall2 = recall2/100

    matriz_precision[i]=precision_recall
    matriz_recall[i]=recall2

  prom_recall=np.mean(matriz_recall,axis=0)
  prom_precision=np.mean(matriz_precision,axis=0)

  return prom_recall, prom_precision

In [8]:
prom_recall_Dab,prom_precision_Dab=recall_precision(relevantes_Dab, num_relevantes_Dab)
prom_recall_Dac,prom_precision_Dac=recall_precision(relevantes_Dac, num_relevantes_Dac)
prom_recall_Dbc,prom_precision_Dbc=recall_precision(relevantes_Dbc, num_relevantes_Dbc)
print("MAP distancia AB:", np.mean(prom_precision_Dab))
print("MAP distancia AC:", np.mean(prom_precision_Dac))
print("MAP distancia BC:", np.mean(prom_precision_Dbc))

MAP distancia AB: 0.9349148711387986
MAP distancia AC: 0.9448283884634097
MAP distancia BC: 0.9118897207207244


####Analizar la efectividad de las distancias combinadas:

Los tres resultados obtenidos de MAP al combinar las distancias de cada algoritmo son mejores que los obtenidos en la tarea anterior donde se calculaba el MAP de las distancias A, B y C por separado.
Esto se puede explicar dado que al combinar las distancias estas se equilibran mutuamente, es decir, si hay una distancia que funciona mejor en un cierto rango de recall y otra que tiene mejores resultados en otro rango, al juntarlas se obtiene una distancia que funciona bien en ambos rangos.

####Repetir el experimento para cuando la matriz combinada $D_{ABC}$ usa las tres matrices de distancia.

In [9]:
D_abc=alg1+alg2+alg3

In [10]:
relevantes_Dabc, num_relevantes_Dabc = rel_numrel(D_abc, clase_query, clase_target)
prom_recall_Dabc, prom_precision_Dabc= recall_precision(relevantes_Dabc, num_relevantes_Dabc)
print("MAP distancia ABC:", np.mean(prom_precision_Dabc))

MAP distancia ABC: 0.9623241240665532


####Analizar la efectividad de las tres distancias combinadas:
El MAP obtenido con la distancia ABC, las tres combinadas, es aproximadamente $0.96$, el que es mejor que las distancias individuales de la tarea anterior, llegando incluso a ser más alto que los tres MAPs de las distancias AB, AC y BC, que dieron como resultado $0.93$, $0.94$ y $0.91$, respectivamente. Por esto, se puede decir, que al combianar más distancias diferentes se obtienes mejores resultados de MAP.

***

##Parte II:
Calculo de distancia con:  $D_{AB}^{\alpha}(i,j) = \alpha \times D_A(i,j) + (1- \alpha) \times D_B(i,j)$

####Calcular $D_{AB}^\alpha$, $D_{AC}^\alpha$ y $D_{BC}^\alpha$ para valores de $\alpha = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]$. Luego, observar cuál es el mejor MAP obtenido según cada distancia.

In [11]:
alphas=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
D_ab_alpha=[]
D_ac_alpha=[]
D_bc_alpha=[]

for a in alphas:
  D_ab_alpha.append(a*alg1+(1.0-a)*alg2)
  D_ac_alpha.append(a*alg1+(1.0-a)*alg3)
  D_bc_alpha.append(a*alg2+(1.0-a)*alg3)

#####MAPs para $D_{AB}^\alpha$:

In [12]:
for i in range(11):
  relevantes, num_relevantes= rel_numrel(D_ab_alpha[i],clase_query,clase_target)
  recall,precision= recall_precision(relevantes, num_relevantes)
  map=np.mean(precision)
  print("MAP distancia AB alpha="+str(alphas[i])+":", map)

MAP distancia AB alpha=0.0: 0.8878501128021675
MAP distancia AB alpha=0.1: 0.9687055442537038
MAP distancia AB alpha=0.2: 0.9673678229653475
MAP distancia AB alpha=0.3: 0.9598543974234357
MAP distancia AB alpha=0.4: 0.9491514694919583
MAP distancia AB alpha=0.5: 0.9349148711387986
MAP distancia AB alpha=0.6: 0.9188950357390382
MAP distancia AB alpha=0.7: 0.9017750617952035
MAP distancia AB alpha=0.8: 0.8817607760997013
MAP distancia AB alpha=0.9: 0.8586656875278077
MAP distancia AB alpha=1.0: 0.8288103601211713


El mejor MAP para la distancia AB es cuando $\alpha=0.1$ y se obtiene un resultado aproximado de $0.968$.

#####MAPs para $D_{AC}^\alpha$:

In [13]:
for i in range(11):
  relevantes, num_relevantes= rel_numrel(D_ac_alpha[i],clase_query,clase_target)
  recall,precision= recall_precision(relevantes, num_relevantes)
  map=np.mean(precision)
  print("MAP distancia AC alpha="+str(alphas[i])+":", map)

MAP distancia AC alpha=0.0: 0.7858305263278307
MAP distancia AC alpha=0.1: 0.9379519875451993
MAP distancia AC alpha=0.2: 0.9479507319166903
MAP distancia AC alpha=0.3: 0.9512382853755621
MAP distancia AC alpha=0.4: 0.9495091317821434
MAP distancia AC alpha=0.5: 0.9448283884634097
MAP distancia AC alpha=0.6: 0.9374172924790165
MAP distancia AC alpha=0.7: 0.927588089255574
MAP distancia AC alpha=0.8: 0.910356330832908
MAP distancia AC alpha=0.9: 0.8825215999064138
MAP distancia AC alpha=1.0: 0.8288103601211713


El valor del MAP más alto es aproximadamente $0.951$ para la distancia AC con $\alpha=0.3$.

#####MAPs para $D_{BC}^\alpha$:

In [14]:
for i in range(11):
  relevantes, num_relevantes= rel_numrel(D_bc_alpha[i],clase_query,clase_target)
  recall,precision= recall_precision(relevantes, num_relevantes)
  map=np.mean(precision)
  print("MAP distancia BC alpha="+str(alphas[i])+":", map)

MAP distancia BC alpha=0.0: 0.7858305263278307
MAP distancia BC alpha=0.1: 0.8385786674283252
MAP distancia BC alpha=0.2: 0.8715156762852303
MAP distancia BC alpha=0.3: 0.8916619720557905
MAP distancia BC alpha=0.4: 0.903679334301323
MAP distancia BC alpha=0.5: 0.9118897207207244
MAP distancia BC alpha=0.6: 0.917117740207344
MAP distancia BC alpha=0.7: 0.9196348873351844
MAP distancia BC alpha=0.8: 0.9188867315185031
MAP distancia BC alpha=0.9: 0.9124238576108593
MAP distancia BC alpha=1.0: 0.8878501128021675


El mejor resultado para la distancia BC se obtiene cuando $\alpha=0.7$ y entrega un MAP aproximado de $0.919$.

####Analizar los resultados:
Se puede observar que cuando $\alpha=0.5$ se obtine los mismos resultados que en la parte I de la tarea, lo cual tiene sentido ya que este alpha provoca que ambas distancias tengan el mismo peso al combinarlas.

Además, se puede ver que los mejores MAPs obtenidos en esta sección son mejores que los resultados de la parte anterior y esto se debe a que el alpha hace que las distancias tengan diferente peso respecto a la otra y no nesariamente es mejor que ambas tengan la misma importancia.

Por lo anterior, hay que tener presente que al combinar distancias se puede tratar de encontrar un alpha que sea más conveniente.

***

##Parte III:

Proponer una combinación lineal de las tres distancias:
$D_{ABC} = \alpha × D_A(i, j) + \beta × D_B(i, j) + \gamma × DC(i, j)$,
tal que $\alpha + \beta + \gamma = 1.0$, y que logre mejorar el MAP obtenido en las otras partes de la
tarea

In [15]:
Dabc=0.2*alg1+0.6*alg2+0.2*alg3
rel_Dabc, num_Dabc =rel_numrel(Dabc,clase_query,clase_target)
recall_Dabc, precision_Dabc=recall_precision(rel_Dabc, num_Dabc)

print("Map distancia ABC:", np.mean(precision_Dabc))

Map distancia ABC: 0.9712616719072106


Con $\alpha=0.2$, $\beta=0.6$ y $\gamma=0.2$, la combinación de las tres distancias obtiene un mejor MAP que todos los anteriormente calculados. Esto concuerda bastante con lo anterior, ya que le da un peso mayor a la distancia B, la que tiene casi siempre los mejores resultados, pero al mezclarla en menor medida con las otras distancias se obtiene el mejor MAP de toda la tarea.

***